# Setup

In [13]:
import collections
import warnings
from time import perf_counter_ns

import numpy as np
import pandas as pd
import xgboost as xgb
from interpret.glassbox import ExplainableBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor

from asboostreg import SparseAdditiveBoostingRegressor

In [14]:
warnings.filterwarnings("ignore")

# Loading the Ames housing dataset

In [15]:
df_housing = pd.read_csv("http://dlsun.github.io/pods/data/AmesHousing.txt", sep="\t")
df_housing.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [16]:
X = df_housing.drop(columns=["SalePrice", "PID", "Order"])
y = df_housing["SalePrice"].astype(float)
y = pd.Series((y - y.mean()) / y.std(), index=y.index)
categorical_indices = np.array(
    [i for i, col in enumerate(X.columns) if X[col].dtype == "object"]
)
X_numeric = X.drop(columns=X.columns[categorical_indices])

In [17]:
X.head()
kf = KFold(n_splits=10, shuffle=True, random_state=0)
cv = list(kf.split(X))

# Defining the models

In [26]:
dummy = DummyRegressor()

# Interpretable but not strong
ridgereg = make_pipeline(
    OneHotEncoder(handle_unknown="ignore", sparse=False, max_categories=10),
    SimpleImputer(add_indicator=True),
    StandardScaler(),
    RidgeCV(cv=cv),
)  # Non Sparse
treereg = make_pipeline(
    OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
    SimpleImputer(add_indicator=True),
    DecisionTreeRegressor(max_depth=4),
)  # Sparse

# Strong but not interpretable
rfreg = make_pipeline(
    OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
    SimpleImputer(add_indicator=True),
    RandomForestRegressor(random_state=0),
)  # Non Sparse
xgbreg = xgb.XGBRegressor(random_state=0)  # Sparse

# Both interpretable and strong
ebmreg = ExplainableBoostingRegressor(interactions=0)  # Non Sparse
sparsereg = SparseAdditiveBoostingRegressor(
    learning_rate=0.01,
    n_estimators=10_000,
    l2_regularization=2.0,
    max_depth=6,
    row_subsample=0.632,
    random_state=0,
    n_iter_no_change=10,
    max_bins=100,
)  # Sparse

# Comparing the models

In [27]:
# Running fast Hyperparameter optimization for Ridge
ridgereg.fit(X_numeric, y)
alpha = ridgereg.named_steps["ridgecv"].alpha_
ridgereg = make_pipeline(
    OneHotEncoder(handle_unknown="ignore", sparse=False, max_categories=10),
    SimpleImputer(add_indicator=True),
    StandardScaler(),
    Ridge(alpha=alpha),
)

In [28]:
def evaluate(model, X_train, X_test, y_train, y_test, **kwargs):
    start = perf_counter_ns()
    model.fit(X_train, y_train, **kwargs)
    end = perf_counter_ns()
    elapsed = (end - start) / 1e9
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    name = (
        model.__class__.__name__
        if not isinstance(model, Pipeline)
        else model.steps[-1][1].__class__.__name__
    )
    print(
        f"{name}: {train_score:.3f} (train), {test_score:.3f} (test), {elapsed:.3f} (s)"
    )
    return test_score

In [29]:
test_scores = collections.defaultdict(list)

for i, (train_index, test_index) in enumerate(cv, 1):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    print(f"Fold {i}")
    print("------")

    test = evaluate(dummy, X_train, X_test, y_train, y_test)
    test_scores["Dummy"].append(test)

    test = evaluate(treereg, X_train, X_test, y_train, y_test)
    test_scores["Decision Tree"].append(test)

    test = evaluate(ridgereg, X_train, X_test, y_train, y_test)
    test_scores["Elastic Net"].append(test)

    X_train_numeric, X_test_numeric = (
        X_numeric.iloc[train_index],
        X_numeric.iloc[test_index],
    )

    test = evaluate(
        xgbreg,
        X_train_numeric,
        X_test_numeric,
        y_train,
        y_test,
        eval_set=[(X_test_numeric, y_test)],
        early_stopping_rounds=30,
        verbose=False,
    )
    test_scores["XGBoost"].append(test)

    test = evaluate(rfreg, X_train, X_test, y_train, y_test)
    test_scores["Random Forest"].append(test)

    test = evaluate(ebmreg, X_train, X_test, y_train, y_test)
    test_scores["EBM"].append(test)

    test = evaluate(
        sparsereg,
        X_train_numeric,
        X_test_numeric,
        y_train,
        y_test,
        validation_set=(X_test_numeric, y_test),
    )
    test_scores["SparseReg"].append(test)

    print()

Fold 1
------
DummyRegressor: 0.000 (train), -0.003 (test), 0.000 (s)
DecisionTreeRegressor: 0.812 (train), 0.554 (test), 0.084 (s)
Ridge: 0.919 (train), 0.812 (test), 0.230 (s)
XGBRegressor: 0.980 (train), 0.885 (test), 0.373 (s)
RandomForestRegressor: 0.984 (train), 0.612 (test), 8.444 (s)
ExplainableBoostingRegressor: 0.950 (train), 0.851 (test), 1.971 (s)
SparseAdditiveBoostingRegressor: -0.291 (train), -0.314 (test), 0.276 (s)

Fold 2
------
DummyRegressor: 0.000 (train), -0.000 (test), 0.000 (s)
DecisionTreeRegressor: 0.808 (train), 0.633 (test), 0.139 (s)
Ridge: 0.915 (train), 0.845 (test), 0.310 (s)
XGBRegressor: 0.984 (train), 0.922 (test), 0.264 (s)


KeyboardInterrupt: 

In [ ]:
test_scores_df = pd.DataFrame(test_scores)
test_scores_df

,Dummy,Decision Tree,Elastic Net,XGBoost,Random Forest,EBM,SparseReg
0,-0.003407,0.553888,0.811892,0.884842,0.611996,0.851345,0.602910
1,-0.000092,0.632983,0.845110,0.921871,0.649268,0.861835,0.635699
2,-0.000387,0.543707,0.844391,0.834974,0.570932,0.880538,-0.396731
3,-0.000486,0.642401,0.893536,0.917660,0.692364,0.925785,0.604775
4,-0.000353,0.649737,0.866149,0.908873,0.650828,0.913224,0.463032
5,-0.000084,0.659418,0.845634,0.913533,0.674692,0.912196,0.546811
6,-0.003682,0.540593,0.752622,0.847567,0.619382,0.819932,0.567193
7,-0.001526,0.697793,0.874272,0.916567,0.722162,0.927635,0.598072
8,-0.006774,0.708505,0.872922,0.922240,0.726452,0.917330,0.629276
9,-0.000190,0.674228,0.873821,0.901770,0.661492,0.893796,-0.160902


In [ ]:
print(test_scores_df.drop("Dummy", axis=1).round(3).to_latex())

\begin{tabular}{lrrrrrr}
\toprule
 & Decision Tree & Elastic Net & XGBoost & Random Forest & EBM & SparseReg \\
\midrule
0 & 0.554000 & 0.812000 & 0.885000 & 0.612000 & 0.851000 & 0.603000 \\
1 & 0.633000 & 0.845000 & 0.922000 & 0.649000 & 0.862000 & 0.636000 \\
2 & 0.544000 & 0.844000 & 0.835000 & 0.571000 & 0.881000 & -0.397000 \\
3 & 0.642000 & 0.894000 & 0.918000 & 0.692000 & 0.926000 & 0.605000 \\
4 & 0.650000 & 0.866000 & 0.909000 & 0.651000 & 0.913000 & 0.463000 \\
5 & 0.659000 & 0.846000 & 0.914000 & 0.675000 & 0.912000 & 0.547000 \\
6 & 0.541000 & 0.753000 & 0.848000 & 0.619000 & 0.820000 & 0.567000 \\
7 & 0.698000 & 0.874000 & 0.917000 & 0.722000 & 0.928000 & 0.598000 \\
8 & 0.709000 & 0.873000 & 0.922000 & 0.726000 & 0.917000 & 0.629000 \\
9 & 0.674000 & 0.874000 & 0.902000 & 0.661000 & 0.894000 & -0.161000 \\
\bottomrule
\end{tabular}

